In [1]:
import logging
import numpy as np

from wave_monitor import WaveMonitor
h = logging.StreamHandler()
h.setLevel(logging.DEBUG)
h.setFormatter(logging.Formatter("%(levelname)s %(message)s"))
WaveMonitor.logger.addHandler(h)

monitor = WaveMonitor()

t = np.linspace(0, 1, 1_000_001)  # 1m pts ~= 1ms for 1GSa/s.
f = 5
i_wave = np.cos(2 * np.pi * f * t)
q_wave = np.sin(2 * np.pi * f * t)

WaveMonitor.logger.setLevel(logging.DEBUG)
monitor.add_wfm("wave_1", t, [i_wave, q_wave])
WaveMonitor.logger.setLevel(logging.INFO)

DEBUG msg to send: {'_type': 'get_wfm_interval'}
DEBUG Wrote 24 bytes, 0 bytes remaining.
DEBUG query received: {'_type': 'wfm_interval', 'interval': 0}
DEBUG Adding waveform 'wave_1'
DEBUG msg to send: {'_type': 'add_wfm', 'name': 'wave_1', 't': array([0.00000e+00, 1.00000e-06, 2.00000e-06, ..., 9.99998e-01,
       9.99999e-01, 1.00000e+00], shape=(1000001,)), 'ys': [array([1., 1., 1., ..., 1., 1., 1.], shape=(1000001,)), array([ 0.00000000e+00,  3.14159265e-05,  6.28318530e-05, ...,
       -6.28318530e-05, -3.14159265e-05, -1.22464680e-15],
      shape=(1000001,))]}
DEBUG Wrote 24000201 bytes, 0 bytes remaining.


In [2]:
%timeit monitor.add_wfm("wave_1", t, [i_wave, q_wave])

32.6 ms ± 2.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
%timeit monitor.get_wfm_interval()

65 ns ± 0.68 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [4]:
mon2 = WaveMonitor()
mon2.get_wfm_interval = lambda: 0  # patch to avoid the delay.
%timeit mon2.add_wfm("wave_2", t, [i_wave, q_wave])

C:\Users\qiujv\dev\WaveMonitor\src\wave_monitor\client.py:203: UserWarning: Monitor is already running, not starting a new one.
  warnings.warn("Monitor is already running, not starting a new one.")


35.4 ms ± 2.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
from PySide6.QtNetwork import QLocalSocket
class FakeSocket:
    def write(self, data: bytes) -> int:
        return len(data)
    
    def waitForBytesWritten(self) -> bool:
        return True
    
    def state(self):
        return QLocalSocket.ConnectedState
    
mon2.sock = FakeSocket()  # patch to avoid the delay.
%timeit mon2.add_wfm("wave_2", t, [i_wave, q_wave])

14.1 ms ± 488 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%timeit monitor.add_wfm("wave_1", t, [i_wave, q_wave])

33.2 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
# close the window now.
%timeit monitor.add_wfm("wave_1", t, [i_wave, q_wave])

C:\Users\qiujv\dev\WaveMonitor\src\wave_monitor\client.py:82: UserWarning: Not connected to server.
  self.write(dict(_type="add_wfm", name=name, t=t, ys=ys))
C:\Users\qiujv\dev\WaveMonitor\src\wave_monitor\client.py:89: UserWarning: Not connected to server.
  msg = self.query(dict(_type="get_wfm_interval"), timeout_ms=200)


12.9 μs ± 140 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [11]:
monitor.refresh_connect()

False

In [12]:
monitor.sock.state()

<LocalSocketState.UnconnectedState: 0>

In [13]:
from wave_monitor import proto
msg = dict(_type="add_wfm", name="wave_1", t=t, ys=[i_wave, q_wave])
payload = proto.encode(msg)
print(f"Payload size: {len(payload)/1e6:.2f} MB")

%timeit proto.encode(msg)

Payload size: 24.00 MB
16.9 ms ± 1.39 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
